__purpose__

    - transform downloaded AdaptWest gridded climate data from Lambert conformal to WGS84
    - then extract climate data within the boundaries of the jack pine range

In [1]:
from pythonimports import *

netdir = '/data/projects/pool_seq/environemental_data/netCDF_files'
netdirs = fs(netdir, dirs=True)

lview,dview = get_client()

56 56


# extract the netCDF files from the 7zip downloaded from AdaptWest

climate data at 1km resolution

https://adaptwest.databasin.org/pages/adaptwest-climatena/

In [2]:
# note py3.7 - this conda env didn't have any conflicts installing command-line gdal
latest_commit()

##################################################################
Current commit of pythonimports:
commit a22cc82763e7d6bc5507200697d8420704934bb7
Author: Brandon <lindb@vcu.edu>
Date:   Thu Jan 28 11:09:19 2021 -0700
Today:	February 01, 2021 - 11:26:27
python version: 3.7.9
##################################################################



In [3]:
# get the 7zip files
netfiles = []
for d in netdirs:
    netfiles.extend(fs(d, endswith='7z'))
netfiles

['/data/projects/pool_seq/environemental_data/netCDF_files/NA_ENSEMBLE_rcp45_2050s/NA_ENSEMBLE_rcp45_2050s_Bioclim_netCDF.7z',
 '/data/projects/pool_seq/environemental_data/netCDF_files/NA_ENSEMBLE_rcp45_2080s/NA_ENSEMBLE_rcp45_2080s_Bioclim_netCDF.7z',
 '/data/projects/pool_seq/environemental_data/netCDF_files/NA_NORM_1961-1990/NA_NORM_1961-1990_netCDF.7z']

In [4]:
# unzip them
for file in netfiles:
    dname = op.dirname(file)
    os.chdir(dname)
    print(ColorText(op.basename(dname)).bold().blue())
    !/data/programs/p7zip_16.02/bin/7z x $file

NA_ENSEMBLE_rcp45_2050s

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,56 CPUs x64)

Scanning the drive for archives:
  0M Scan /data/projects/pool_seq/environ . F_files/NA_ENSEMBLE_rcp45_2050                                                                          1 file, 316949670 bytes (303 MiB)

Extracting archive: /data/projects/pool_seq/environemental_data/netCDF_files/NA_ENSEMBLE_rcp45_2050s/NA_ENSEMBLE_rcp45_2050s_Bioclim_netCDF.7z
--
Path = /data/projects/pool_seq/environemental_data/netCDF_files/NA_ENSEMBLE_rcp45_2050s/NA_ENSEMBLE_rcp45_2050s_Bioclim_netCDF.7z
Type = 7z
Physical Size = 316949670
Headers Size = 568
Method = LZMA2:26
Solid = +
Blocks = 2

      0% - AHM.n              1% - AHM.n              2% - AHM.n              3% - AHM.n              3% 1 - bFFP.                4% 1 - bFFP.                5% 1 - bFFP.                6% 1 - bFFP.                7% 1 - bFFP.          

# convert each netCDF file from Lambert Conformal to WGS84

In [5]:
# original proj string as-downloaded (the coordinate projection of the data)
# technically `+lon_0=-95.0` was `+lon_0=--95.0` but `--` didn't make any sense
proj = '+proj=lcc +lat_1=49.0 +lat_2=77.0 +lat_0=0.0 +lon_0=-95.0 +x_0=0.0 +y_0=0.0 +ellps=WGS84 \
+ datum=WGS84 +units=m +no_defs'

In [6]:
def gdalwarp(netcdf_infile, netcdf_outfile, proj):
    """Convert netcdf_infile to WGS84 netcdf_outfile.
    
    Notes
    -----
    conda install -c conda-forge gdal 
    """
    import subprocess, shutil
    
    output = subprocess.check_output([shutil.which('gdalwarp'),
                                      '-s_srs', proj,
                                      '-t_srs',  '+proj=longlat +ellps=WGS84',
                                      '-of', 'netCDF',
                                      netcdf_infile,
                                      netcdf_outfile,
                                      '-overwrite']).decode('utf-8').split('\n')
    return output

In [7]:
# get the netCDF files
netfiles = []
for d in netdirs:
    netfiles.extend(fs(d, endswith='.nc', exclude='WGS84'))
print(len(netfiles))
netfiles[0]

81


'/data/projects/pool_seq/environemental_data/netCDF_files/NA_ENSEMBLE_rcp45_2050s/AHM.nc'

In [8]:
# test
gdalwarp(netfiles[0], netfiles[0].replace('.nc', '_WGS84.nc'), proj)

['Creating output file that is 15399P x 3236L.',
 'Processing /data/projects/pool_seq/environemental_data/netCDF_files/NA_ENSEMBLE_rcp45_2050s/AHM.nc [1/1] : 0Using internal nodata values (e.g. -3.4e+38) for image /data/projects/pool_seq/environemental_data/netCDF_files/NA_ENSEMBLE_rcp45_2050s/AHM.nc.',
 'Copying nodata values from source /data/projects/pool_seq/environemental_data/netCDF_files/NA_ENSEMBLE_rcp45_2050s/AHM.nc to destination /data/projects/pool_seq/environemental_data/netCDF_files/NA_ENSEMBLE_rcp45_2050s/AHM_WGS84.nc.',
 '...10...20...30...40...50...60...70...80...90...100 - done.',
 '']

In [9]:
# run in parallel
jobs = []
for netfile in netfiles:
    jobs.append(lview.apply_async(gdalwarp, *(netfile, netfile.replace('.nc', '_WGS84.nc'), proj)))
watch_async(jobs, desc='gdalwarp')


Watching 81 jobs ...


gdalwarp: 100%|██████████| 81/81 [00:21<00:00,  3.81it/s]


In [10]:
# make sure there weren't any errors
count = 0
for j in jobs:
    try:
        x = j.r
    except:
        count += 1
count

0

In [11]:
# double check files were made
outfiles = []
for d in netdirs:
    outfiles.extend(fs(d, 'WGS84', endswith='.nc'))
assert len(outfiles) == len(netfiles)

# crop each WGS84 netCDF file using range map for use in future projections

In [2]:
# back to py38
latest_commit()

##################################################################
Current commit of pythonimports:
commit a22cc82763e7d6bc5507200697d8420704934bb7
Author: Brandon <lindb@vcu.edu>
Date:   Thu Jan 28 11:09:19 2021 -0700
Today:	February 01, 2021 - 13:04:46
python version: 3.8.5
##################################################################



In [3]:
def clip(netcdf_file, outfile, shapefile='/data/projects/pool_seq/environemental_data/shapefiles/jackpine.shp'):
    """Clip netCDF file within shapefile boundaries, save as tif and dataframe with coords."""
    import geopandas, rioxarray, cartopy.crs, xarray
    from shapely.geometry import box, mapping

    # read in netcdf file
    dataset = xarray.open_dataset(netcdf_file)
    # add metadata for WGS84
    dataset.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
    dataset.rio.write_crs("epsg:4326", inplace=True)

    # read in the shapefile
    shp = geopandas.read_file(shapefile, crs=cartopy.crs.PlateCarree())
    
    # clip climate data within boundaries of shapefile
    clipped = dataset.rio.clip(shp.geometry.apply(mapping), shp.crs, drop=False)
    
    # save as .tif
    tif = outfile.replace('.txt', '.tif')
    clipped.rio.to_raster(tif)
    
    # convert to dataframe, remove null points
    df = clipped.to_dataframe()
    df = df[df['Band1'].notnull()]
    # convert multi-index (lat/long) to column data
    df.reset_index(drop=False, inplace=True)
    
    # save
    df.to_csv(outfile, sep='\t', index=False)
    
    return outfile, tif
    

In [4]:
# refind the outfiles (new session)
outfiles = []
for d in netdirs:
    outfiles.extend(fs(d, 'WGS84', endswith='.nc'))
len(outfiles)    

81

In [5]:
# test
clip(outfiles[0], outfiles[0].replace('.nc', '_clipped_jack-pine.txt'))

('/data/projects/pool_seq/environemental_data/netCDF_files/NA_ENSEMBLE_rcp45_2050s/AHM_WGS84_clipped_jack-pine.txt',
 '/data/projects/pool_seq/environemental_data/netCDF_files/NA_ENSEMBLE_rcp45_2050s/AHM_WGS84_clipped_jack-pine.tif')

In [6]:
# run in parallel 
jobs = []
for f in outfiles:
    jobs.append(lview.apply_async(clip, *(f, f.replace('.nc', '_clipped_jack-pine.txt'))))
watch_async(jobs, desc='clip')


Watching 81 jobs ...


clip: 100%|██████████| 81/81 [00:40<00:00,  2.00it/s]


In [7]:
# get the txt files
outs = {}
for d in netdirs:
    outs[op.basename(d)] = fs(d, endswith='.txt')
assert len(flatten(outs.values())) == len(outfiles)

In [8]:
def read_file(file):
    import pandas
    import os
    env = os.path.basename(file).split("_")[0]
    df = pandas.read_table(file)
    df.pop('crs')
    df = df.set_index(['lat','lon'])
    df.columns = [env]
    return df

In [9]:
# test
df1 = read_file(outs[op.basename(d)][0])
df1.head()

AHM
lat       lon             
41.488534 -87.410886  21.0
          -87.387508  21.1
          -87.364130  21.1
          -87.340752  21.1
          -87.317374  21.1

In [11]:
# combine txt files into single dataframe
dfs = {}
for dname,files in outs.items():
    print(ColorText(f'\n{dname}').bold().blue())
    jobs = []
    for f in files:
        jobs.append(lview.apply_async(read_file, f))
    watch_async(jobs, desc=dname)
    dfs[dname] = pd.concat([j.r for j in jobs], axis=1)
    display(dfs[dname].head())


NA_ENSEMBLE_rcp45_2050s

Watching 27 jobs ...


NA_ENSEMBLE_rcp45_2050s: 100%|██████████| 27/27 [00:01<00:00, 13.95it/s]


AHM    CMD   DD18     DD5     DD      DD   EMT   EXT  \
lat       lon                                                                 
41.488534 -87.410886  22.9  246.0  907.0  3565.0  229.0  2695.0 -26.0  42.8   
          -87.387508  23.0  249.0  909.0  3570.0  228.0  2690.0 -26.0  42.8   
          -87.364130  23.0  252.0  911.0  3573.0  227.0  2686.0 -26.0  42.9   
          -87.340752  23.0  252.0  912.0  3577.0  225.0  2680.0 -25.9  42.9   
          -87.317374  23.0  253.0  908.0  3571.0  226.0  2684.0 -26.0  42.8   

                        Eref    FFP     MAP   MAR   MAT  MCMT    MSP  MWMT  \
lat       lon                                                                
41.488534 -87.410886  1015.0  190.0  1005.0  14.5  13.0  -1.7  485.0  26.6   
          -87.387508  1018.0  190.0  1003.0  14.5  13.0  -1.6  484.0  26.6   
          -87.364130  1019.0  190.0  1002.0  14.5  13.1  -1.6  483.0  26.6   
          -87.340752  1021.0  190.0  1002.0  14.5  13.1  -1.5  482.0  26.6   
          -87.317374  1020.0  190.0  1002.0  14.5  13.1  -1.5  482.0  26.6   

                       NFFD   PAS    PPT    PPT    RH   SHM    TD  Tave  Tave  \
lat       lon                                                                   
41.488534 -87.410886  227.0  26.0  288.0  167.0  65.0  54.8  28.3  25.5  -0.2   
          -87.387508  227.0  26.0  287.0  167.0  65.0  54.9  28.2  25.5  -0.2   
          -87.364130  227.0  25.0  285.0  167.0  65.0  55.1  28.2  25.5  -0.2   
          -87.340752  227.0  25.0  284.0  168.0  65.0  55.3  28.2  25.5  -0.1   
          -87.317374  227.0  26.0  284.0  169.0  65.0  55.2  28.1  25.5  -0.1   

                       bFFP   eFFP  
lat       lon                       
41.488534 -87.410886  107.0  297.0  
          -87.387508  107.0  297.0  
          -87.364130  107.0  297.0  
          -87.340752  107.0  297.0  
          -87.317374  108.0  297.0


NA_ENSEMBLE_rcp45_2080s

Watching 27 jobs ...


NA_ENSEMBLE_rcp45_2080s: 100%|██████████| 27/27 [00:01<00:00, 24.56it/s] 


AHM    CMD   DD18     DD5     DD      DD   EMT   EXT  \
lat       lon                                                                 
41.488534 -87.410886  23.3  259.0  994.0  3724.0  195.0  2559.0 -25.1  43.4   
          -87.387508  23.3  264.0  996.0  3729.0  193.0  2554.0 -25.1  43.4   
          -87.364130  23.4  265.0  997.0  3732.0  192.0  2550.0 -25.1  43.4   
          -87.340752  23.4  267.0  999.0  3737.0  190.0  2544.0 -25.0  43.5   
          -87.317374  23.4  268.0  995.0  3730.0  191.0  2548.0 -25.1  43.4   

                        Eref    FFP     MAP   MAR   MAT  MCMT    MSP  MWMT  \
lat       lon                                                                
41.488534 -87.410886  1037.0  195.0  1015.0  14.5  13.6  -0.8  486.0  27.3   
          -87.387508  1040.0  195.0  1013.0  14.6  13.7  -0.7  485.0  27.3   
          -87.364130  1042.0  196.0  1012.0  14.6  13.7  -0.7  484.0  27.3   
          -87.340752  1043.0  196.0  1012.0  14.6  13.7  -0.6  482.0  27.3   
          -87.317374  1043.0  195.0  1012.0  14.6  13.7  -0.6  482.0  27.3   

                       NFFD   PAS    PPT    PPT    RH   SHM    TD  Tave  Tave  \
lat       lon                                                                   
41.488534 -87.410886  233.0  20.0  291.0  171.0  65.0  56.0  28.0  26.1   0.5   
          -87.387508  234.0  19.0  289.0  171.0  65.0  56.2  28.0  26.1   0.5   
          -87.364130  234.0  19.0  288.0  172.0  65.0  56.4  28.0  26.1   0.5   
          -87.340752  234.0  19.0  287.0  173.0  65.0  56.6  27.9  26.1   0.6   
          -87.317374  234.0  19.0  286.0  173.0  65.0  56.5  27.9  26.1   0.6   

                       bFFP   eFFP  
lat       lon                       
41.488534 -87.410886  104.0  300.0  
          -87.387508  104.0  300.0  
          -87.364130  104.0  300.0  
          -87.340752  104.0  300.0  
          -87.317374  105.0  300.0


NA_NORM_1961-1990

Watching 27 jobs ...


NA_NORM_1961-1990: 100%|██████████| 27/27 [00:01<00:00, 21.56it/s] 


AHM    CMD   DD18     DD5     DD      DD   EMT   EXT  \
lat       lon                                                                 
41.488534 -87.410886  21.0  174.0  504.0  2839.0  444.0  3376.0 -30.3  39.6   
          -87.387508  21.1  179.0  505.0  2843.0  441.0  3370.0 -30.3  39.6   
          -87.364130  21.1  181.0  506.0  2846.0  439.0  3366.0 -30.3  39.6   
          -87.340752  21.1  183.0  508.0  2850.0  437.0  3360.0 -30.2  39.6   
          -87.317374  21.1  183.0  504.0  2844.0  438.0  3365.0 -30.2  39.6   

                       Eref    FFP    MAP   MAR   MAT  MCMT    MSP  MWMT  \
lat       lon                                                              
41.488534 -87.410886  879.0  168.0  955.0  14.1  10.1  -4.9  488.0  23.3   
          -87.387508  882.0  168.0  953.0  14.1  10.1  -4.9  487.0  23.4   
          -87.364130  884.0  168.0  952.0  14.1  10.1  -4.8  486.0  23.4   
          -87.340752  885.0  168.0  952.0  14.1  10.1  -4.8  485.0  23.4   
          -87.317374  884.0  168.0  952.0  14.1  10.1  -4.8  485.0  23.3   

                       NFFD   PAS    PPT    PPT    RH   SHM    TD  Tave  Tave  \
lat       lon                                                                   
41.488534 -87.410886  208.0  74.0  297.0  152.0  64.0  47.8  28.3  22.3  -3.4   
          -87.387508  207.0  74.0  295.0  152.0  64.0  47.9  28.2  22.3  -3.4   
          -87.364130  207.0  73.0  294.0  152.0  64.0  48.1  28.2  22.3  -3.3   
          -87.340752  207.0  73.0  292.0  153.0  64.0  48.3  28.2  22.3  -3.3   
          -87.317374  207.0  74.0  292.0  153.0  64.0  48.2  28.1  22.3  -3.3   

                       bFFP   eFFP  
lat       lon                       
41.488534 -87.410886  120.0  288.0  
          -87.387508  120.0  288.0  
          -87.364130  120.0  288.0  
          -87.340752  120.0  288.0  
          -87.317374  120.0  288.0

In [12]:
# save 
for dname,df in dfs.items():
    print(ColorText(dname).bold().blue())
    f = op.join(netdir, f'{dname}/{dname}_all-envs_WGS84_clipped_jack-pine.txt')
    df.reset_index(drop=False, inplace=True)
    df.to_csv(f, sep='\t', index=False)
df.head()

NA_ENSEMBLE_rcp45_2050s
NA_ENSEMBLE_rcp45_2080s
NA_NORM_1961-1990


,lat,lon,AHM,CMD,DD18,DD5,DD,DD,EMT,EXT,Eref,FFP,MAP,MAR,MAT,MCMT,MSP,MWMT,NFFD,PAS,PPT,PPT,RH,SHM,TD,Tave,Tave,bFFP,eFFP
0,41.488534,-87.410886,21.0,174.0,504.0,2839.0,444.0,3376.0,-30.3,39.6,879.0,168.0,955.0,14.1,10.1,-4.9,488.0,23.3,208.0,74.0,297.0,152.0,64.0,47.8,28.3,22.3,-3.4,120.0,288.0
1,41.488534,-87.387508,21.1,179.0,505.0,2843.0,441.0,3370.0,-30.3,39.6,882.0,168.0,953.0,14.1,10.1,-4.9,487.0,23.4,207.0,74.0,295.0,152.0,64.0,47.9,28.2,22.3,-3.4,120.0,288.0
2,41.488534,-87.364130,21.1,181.0,506.0,2846.0,439.0,3366.0,-30.3,39.6,884.0,168.0,952.0,14.1,10.1,-4.8,486.0,23.4,207.0,73.0,294.0,152.0,64.0,48.1,28.2,22.3,-3.3,120.0,288.0
3,41.488534,-87.340752,21.1,183.0,508.0,2850.0,437.0,3360.0,-30.2,39.6,885.0,168.0,952.0,14.1,10.1,-4.8,485.0,23.4,207.0,73.0,292.0,153.0,64.0,48.3,28.2,22.3,-3.3,120.0,288.0
4,41.488534,-87.317374,21.1,183.0,504.0,2844.0,438.0,3365.0,-30.2,39.6,884.0,168.0,952.0,14.1,10.1,-4.8,485.0,23.3,207.0,74.0,292.0,153.0,64.0,48.2,28.1,22.3,-3.3,120.0,288.0


# get a set of loci

In [89]:
pklload = timer(pklload)
DIR = '/data/projects/pool_seq/pangenome/JP_pangenome/JP_pooled/snpsANDindels/03_maf-p05_RD-recalculated/baypass/final_results'
pkl = op.join(DIR, 'envdfs_after_ranking.pkl')
baypass = pklload(pkl)

Function `pklload` completed after : 0-00:00:16
Function `pklload` completed after : 0-00:00:16


In [70]:
transpkl = pklload(op.join(DIR.split('baypass')[0], 'translated_loci_dict.pkl'))
len(transpkl)

Function `pklload` completed after : 0-00:00:01


1235752

In [90]:
loci = []
for env,df in pbar(baypass.items()):
    df2 = df[df['mean_BF(dB)']>=75]
#     loci.extend(pd.Series(df2.index).map(transpkl).tolist())
    loci.extend(df2.index.tolist())
len(loci)

100%|██████████| 19/19 [00:00<00:00, 202.11it/s]


5561

In [91]:
df = pd.DataFrame(loci)
df.columns = ['baypass_outliers']
df.head()

,baypass_outliers
0,Scaffold_162-7034260
1,Scaffold_162-7878981
2,Scaffold_481-2420825
3,Scaffold_606-1589029
4,Scaffold_1858-354838


In [92]:
df.to_csv('~/data/testdir/jack_pine_baypass_gt75_loci2.txt', sep='\t', index=False)